In [5]:
import sqlite3

conn = sqlite3.connect('./quant_db.sqlite', isolation_level= None)
c = conn.cursor()

In [6]:
import pandas as pd

ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
and 종목구분 = '보통주'
and 시장구분 = 'KOSPI';
""", con=conn)

ticker_list.shape

(806, 11)

In [7]:
ticker_list.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4460.0,2.088274e+11,2023-09-19,201.0,593.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18640.0,2.469345e+11,2023-09-19,NaN,NaN,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3625.0,3.469734e+11,2023-09-19,247.0,650.0,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,149200.0,2.578759e+12,2023-09-19,11203.0,13573.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7210.0,2.322258e+12,2023-09-19,2404.0,2553.0,30468.0,625.0,보통주


In [8]:
def filtering_data_that_market_cap_under(df: pd.DataFrame, cap_size=0.2) -> pd.DataFrame:
    """
    코스피, 코스닥의 종목에서 시가총액 30%이하의 종목으로 필터함.
    이 때, 기업소재지가 외국, 스팩주, 우선주, 최신거래일에 거래량이 0인 종목은 제거함.
    :param data:
    :return: DataFrame
    """
    df["시가총액"] = df["시가총액"] / 100000000
    return df[df["시가총액"] <= df["시가총액"].quantile(q=cap_size)].sort_values(by=["시가총액"], ascending=True)

CAP_SIZE = 0.2
small_companies = filtering_data_that_market_cap_under(ticker_list, CAP_SIZE)
print("시가총액 Max: ", small_companies["시가총액"].max())
print("기업 수: ", small_companies.shape[0])

시가총액 Max:  884.04
기업 수:  162


In [9]:
price_list = pd.read_sql("""
select * from kor_price
where 날짜 = (select max(날짜) from kor_price) ;
""", con=conn)

# price_list.head()

small_companies = small_companies.merge(price_list,how='left',on='종목코드')
small_companies = small_companies[['종목코드','종목명','시가총액','기준일','주당배당금','거래량']]
small_companies.head()

,종목코드,종목명,시가총액,기준일,주당배당금,거래량
0,008500,일정실업,154.800000,2023-09-19,0.0,0.0
1,071950,코아스,185.134188,2023-09-19,0.0,44753.0
2,030790,비케이탑스,192.083231,2023-09-19,0.0,0.0
3,088790,진도,277.584691,2023-09-19,70.0,20613.0
4,001770,SHD,280.029379,2023-09-19,100.0,1406.0


In [10]:
def remove_bad_companies(df: pd.DataFrame) -> None:
    # 스팩 주식 드랍
    df.drop(
        df[df["종목명"].str.contains("스팩")].index,
        inplace=True
    )
    # 우선주 드랍
    df.drop(
        df[df["종목명"].str.endswith(("우", "우B", "우C"))].index,
        inplace=True
    )
    # 지주사 드랍
    df.drop(
        df[df["종목명"].str.endswith(("홀딩스", "지주", "지주회사"))].index,
        inplace=True
    )
    # 직전 거래일의 거래량이 0인 경우는 어떠한 이유에서 거래정지가 되어있을 확률이 높음
    df = df[df["거래량"] > 0]
    return df

small_companies = remove_bad_companies(small_companies)
print("그나마 멀쩡한 기업 수: ", small_companies.shape[0])

그나마 멀쩡한 기업 수:  145


In [11]:
import numpy as np

value_list = pd.read_sql("""
select * from kor_value
where 기준일 = (select max(기준일) from kor_value);
""", con=conn)

# 가치 지표가 음수인 기없은 계산 안하기 위해 (마지막으로 망할 것 같은 기업 제거 수단)
value_list.loc[value_list['값'] <= 0, '값'] = np.nan

value_pivot = value_list.pivot(index='종목코드', columns='지표', values='값')
small_companies = small_companies[['종목코드', '종목명','시가총액']].merge(value_pivot, how='left', on='종목코드')
small_companies.head()

,종목코드,종목명,시가총액,DY,PBR,PCR,PER,PSR
0,071950,코아스,185.134188,NaN,1.0343,NaN,NaN,0.2291
1,088790,진도,277.584691,0.0314,0.2689,3.4698,7.5023,0.4573
2,001770,SHD,280.029379,0.0043,0.5491,NaN,4.5906,0.2502
3,009310,참엔지니어링,293.159747,NaN,0.3354,NaN,NaN,0.2124
4,010420,한솔PNS,294.689513,0.0174,0.5818,NaN,NaN,0.0944


In [12]:
print("기업 수: ", small_companies.shape[0])

기업 수:  145


In [13]:
fs_list = pd.read_sql("""
select * from kor_fs
where 계정 in ('당기순이익', '매출총이익', '영업활동으로인한현금흐름', '자산', '자본')
and 공시구분 = 'q';
""", con=conn)

fs_list = fs_list.sort_values(['종목코드', '계정', '기준일'])
fs_list['ttm'] = fs_list.groupby(['종목코드', '계정'], as_index=False)['값'].rolling(window=4, min_periods=4).sum()['값']

fs_list_clean = fs_list.copy()
fs_list_clean['ttm'] = np.where(fs_list_clean['계정'].isin(['자산', '지배기업주주지분']),fs_list_clean['ttm'] / 4, fs_list_clean['ttm'])
fs_list_clean = fs_list_clean.groupby(['종목코드', '계정']).tail(1)

fs_list_pivot = fs_list_clean.pivot(index='종목코드', columns='계정', values='ttm')
fs_list_pivot['ROE'] = fs_list_pivot['당기순이익'] / fs_list_pivot['자본']
fs_list_pivot['GPA'] = fs_list_pivot['매출총이익'] / fs_list_pivot['자산']
fs_list_pivot['CFO'] = fs_list_pivot['영업활동으로인한현금흐름'] / fs_list_pivot['자산']

small_companies = small_companies.merge(fs_list_pivot, how='left', on='종목코드')
small_companies.head()

,종목코드,종목명,시가총액,DY,PBR,PCR,PER,PSR,당기순이익,매출총이익,영업활동으로인한현금흐름,자본,자산,ROE,GPA,CFO
0,071950,코아스,185.134188,NaN,1.0343,NaN,NaN,0.2291,-34.0,208.0,-4.0,716.0,743.50,-0.047486,0.279758,-0.005380
1,088790,진도,277.584691,0.0314,0.2689,3.4698,7.5023,0.4573,37.0,278.0,80.0,4129.0,1279.75,0.008961,0.217230,0.062512
2,001770,SHD,280.029379,0.0043,0.5491,NaN,4.5906,0.2502,61.0,121.0,-16.0,2040.0,882.75,0.029902,0.137072,-0.018125
3,009310,참엔지니어링,293.159747,NaN,0.3354,NaN,NaN,0.2124,-197.0,241.0,-120.0,3496.0,7392.50,-0.056350,0.032601,-0.016233
4,010420,한솔PNS,294.689513,0.0174,0.5818,NaN,NaN,0.0944,-17.0,293.0,-90.0,2026.0,1276.50,-0.008391,0.229534,-0.070505


In [14]:
print("기업 수: ", small_companies.shape[0])

기업 수:  145


In [15]:
# Last filter

# small_companies[ ~small_companies['PBR'].isna() & ~small_companies['PER'].isna() & \
#                  ~small_companies['PCR'].isna() & ~small_companies['PSR'].isna()].shape

small_companies = small_companies[ ~small_companies['PER'].isna()]
small_companies = small_companies[ small_companies['PBR'] >= 0.2]
small_companies.shape

(71, 16)

In [23]:
value_rank = small_companies[['PBR','PCR','PSR']].rank(axis = 0, ascending=True)
value_rank[['GPA', 'ROE', 'CFO']] = small_companies[['GPA', 'ROE', 'CFO']].rank(axis = 0, ascending=False)
value_sum = value_rank.sum(axis = 1, skipna = False).rank()
small_companies.loc[value_sum <= 20, ['종목코드', '종목명', 'PBR', 'GPA']].head(30)

,종목코드,종목명,PBR,GPA
1,088790,진도,0.2689,0.217230
37,005320,국동,0.4130,0.245448
59,123700,SJM,0.3213,0.184255
64,069730,DSR제강,0.3533,0.163745
70,001620,케이비아이동국실업,0.3648,0.129315
75,009180,한솔로지스틱스,0.6317,0.271919
81,129260,인터지스,0.2788,0.121932
84,005870,휴니드,0.5161,0.105751
88,378850,화승알앤에이,0.7855,0.274397
90,155660,DSR,0.3548,0.157046


In [22]:
value_rank = small_companies[['PBR']].rank(axis = 0, ascending=True)
value_rank[['GPA']] = small_companies[['GPA']].rank(axis = 0, ascending=False)
value_sum = value_rank.sum(axis = 1, skipna = False).rank()
small_companies.loc[value_sum <= 20, ['종목코드', '종목명', 'PBR', 'GPA']].head(30)

,종목코드,종목명,PBR,GPA
1,088790,진도,0.2689,0.217230
8,003680,한성기업,0.6008,0.213737
32,084870,TBH글로벌,0.7258,1.050496
37,005320,국동,0.4130,0.245448
57,009770,삼정펄프,0.2895,0.086180
58,005360,모나미,0.5900,0.236768
59,123700,SJM,0.3213,0.184255
64,069730,DSR제강,0.3533,0.163745
70,001620,케이비아이동국실업,0.3648,0.129315
72,006980,우성,0.3987,0.188489
